In [17]:
#Libraries
import numpy as np

import pandas as pd
import glob
import os
import re
import pickle



In [19]:
# get the most recent data of the current season, as the csv cannot be used as a whole (all season's games for each team
# except the last 5: (--> whyy?)

current_season_df = pd.read_csv('../data/current_season/buli_24_25.csv')

# i used this in past model's training, but its not necessary to cutg out the last 5 matches. the question to me now is rather
#whether to use the first 5 matches of a season
# take out the past 5 matches per team, so the last 45 games:
#current_season_beginning = current_season_df.iloc[:-45]
#current_season_beginning.to_csv('../data/buli_24_25_exc_last_5.csv')

#hence bring in current_season_df to the other seasons:
current_season_df.to_csv('../data/buli_24_25.csv')

In [21]:
# Specify the folder path where CSV files are stored
folder_path = '../data/'

# Use glob to find all CSV files in the specified folder
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Use a list comprehension to read each CSV file into a DataFrame and ensure 'Date' is string
df_list = []
for file in all_files:
    try:
        # Read each CSV and convert 'Date' to string format
        buli_df = pd.read_csv(file, encoding='ISO-8859-1', dtype={'Date': str})
        df_list.append(buli_df)
    except pd.errors.ParserError as e:
        print(f"ParserError parsing {file}: {e}")
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in {file}: {e}")

# Concatenate all DataFrames in the list into a single DataFrame
buli_df = pd.concat(df_list, ignore_index=True)

# Standardize and parse the 'Date' column
buli_df['Date'] = buli_df['Date'].str.strip()  # Remove extra whitespace
buli_df['Date'] = buli_df['Date'].replace(r'[/-]', '-', regex=True)  # Replace separators with '-'

# Attempt to parse dates as `dayfirst` and handle both `dd/mm/yyyy` and `dd/mm/yy`
buli_df['Date'] = buli_df['Date'].apply(lambda x: re.sub(r'(\d{2}/\d{2}/)(\d{2})$', r'\120\2', x))
buli_df['Date'] = pd.to_datetime(buli_df['Date'], dayfirst=True, errors='coerce')

# Check for any remaining NaT values in 'Date' after parsing
missing_dates = buli_df[buli_df['Date'].isna()]
if not missing_dates.empty:
    print("Warning: Some dates could not be parsed after concatenation.")
    print(missing_dates)

# Sort by date
buli_df = buli_df.sort_values(['Date']).reset_index(drop=True)

# Display the combined DataFrame
print("Final combined DataFrame with parsed dates:")
buli_df

Final combined DataFrame with parsed dates:


/var/folders/72/b7zxktp96cz1n4tjk3mlbj5w0000gn/T/ipykernel_2962/3583668836.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  buli_df['Date'] = pd.to_datetime(buli_df['Date'], dayfirst=True, errors='coerce')


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BFECAHA,IWCH,IWCD,IWCA,VCCH,VCCD,VCCA,SJH,SJD,SJA
0,D1,2014-08-22,Bayern Munich,Wolfsburg,2,1,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.25,5.8,9.00
1,D1,2014-08-23,Hoffenheim,Augsburg,2,0,H,2,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.91,3.4,3.75
2,D1,2014-08-23,Hertha,Werder Bremen,2,2,D,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.95,3.4,3.75
3,D1,2014-08-23,Dortmund,Leverkusen,0,2,A,0,1,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.57,4.0,5.25
4,D1,2014-08-23,FC Koln,Hamburg,0,0,D,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.95,3.4,3.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,D1,2024-12-21,Holstein Kiel,Augsburg,5,1,H,4,1,H,...,1.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3191,D1,2024-12-21,Stuttgart,St Pauli,0,1,A,0,1,A,...,1.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3192,D1,2024-12-21,Ein Frankfurt,Mainz,1,3,A,0,2,A,...,2.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3193,D1,2024-12-22,Bochum,Heidenheim,2,0,H,2,0,H,...,2.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
#checking for null values
buli_df.isna().sum()
#dropping rows & columns with all null values
buli_df.dropna(axis=1, how='all', inplace=True) #dropped 3 columns
buli_df.dropna(axis=0, how='all',inplace=True) #0 rows dropped

In [25]:
buli_df_red = buli_df[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']]

df = buli_df_red

df.columns = ['date', 'home_team', 'away_team', 'fthg', 'ftag', 'ftr', 'hs', 'as', 'hst', 'ast', 'hf', 'af', 'hc', 'ac', 'hy', 'ay', 'hr', 'ar']

In [27]:
# importing webscraped advanced stats csv
adv_df = pd.read_csv('../data/other_stats/adv_stats.csv')
# transform object-typed date column to datetime
adv_df['Date'] = pd.to_datetime(adv_df['Date'])
#change column names
adv_df.columns = ['date', 'home_team', 'away_team', 'xg_home', 'xg_away', 'deep_home', 'deep_away', 'ppda_home', 'ppda_away', 'xpts_home', 'xpts_away']

In [29]:
# match team names of both dfs
name_mapping = {
    'Borussia Dortmund': 'Dortmund',
    'Borussia M.Gladbach': "M'gladbach",
    'Bayer Leverkusen': 'Leverkusen',
    'Mainz': 'Mainz 05',
    'Ein Frankfurt': 'Eintracht Frankfurt',
    'FC Koln': '1.FC Köln',
    'FC Cologne': '1.FC Köln',
    'Hertha': 'Hertha BSC',
    'Hertha Berlin': 'Hertha BSC',
    'Stuttgart': 'VfB Stuttgart',
    'RB Leipzig': 'Leipzig',
    'RasenBallsport Leipzig': 'Leipzig',
    'Hamburg': 'Hamburger SV',
    'Hannover': 'Hannover 96',
    'Bochum': 'VfL Bochum',
    'Fortuna Dusseldorf': 'Fortuna Düsseldorf',
    'Fortuna Duesseldorf': 'Fortuna Düsseldorf',
    'Bielefeld': 'Arminia Bielefeld',
    'FC Heidenheim': 'Heidenheim',
    'Nuernberg': '1.FC Nürnberg',
    'Nurnberg': '1.FC Nürnberg',
    'Greuther Furth': 'Greuther Fürth',
    'Greuther Fuerth': 'Greuther Fürth',
    'St Pauli': 'St. Pauli'
}

df['home_team'].replace(name_mapping, inplace=True)
df['away_team'].replace(name_mapping, inplace=True)
adv_df['home_team'].replace(name_mapping, inplace=True)
adv_df['away_team'].replace(name_mapping, inplace=True)
print(df['home_team'].value_counts())
print(adv_df['away_team'].value_counts())

home_team
Wolfsburg              178
Hoffenheim             178
Dortmund               178
Leverkusen             178
M'gladbach             178
Mainz 05               178
Augsburg               178
Bayern Munich          177
Eintracht Frankfurt    177
Werder Bremen          160
Freiburg               160
1.FC Köln              153
Hertha BSC             153
VfB Stuttgart          144
Leipzig                143
Schalke 04             136
Union Berlin            92
Hamburger SV            68
Hannover 96             68
VfL Bochum              58
Darmstadt               51
Ingolstadt              34
Paderborn               34
Fortuna Düsseldorf      34
Arminia Bielefeld       34
Heidenheim              24
1.FC Nürnberg           17
Greuther Fürth          17
Holstein Kiel            8
St. Pauli                7
Name: count, dtype: int64
away_team
Bayern Munich          178
Eintracht Frankfurt    178
Mainz 05               177
M'gladbach             177
Augsburg               177
Hoffenhei

/var/folders/72/b7zxktp96cz1n4tjk3mlbj5w0000gn/T/ipykernel_2962/381906689.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['home_team'].replace(name_mapping, inplace=True)
/var/folders/72/b7zxktp96cz1n4tjk3mlbj5w0000gn/T/ipykernel_2962/381906689.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['home_team'].replace(name_mapping, i

In [31]:
#merge dfs

merged_df = pd.merge(df, adv_df, on=['date', 'home_team', 'away_team'], how='inner')

In [33]:
csv_file_path = "../data/prepared_data/basic_plus_adv_stats.csv"
merged_df.to_csv(csv_file_path, index=False)